In [2]:
from tqdm import tqdm
from utils import set_logger
from dataset import FinedustDataset
from model import FinedustLSTM
from utils import prepare_data
from sklearn.model_selection import train_test_split
import os
import logging
from interpolate import simple_interpolate
import numpy as np
from preprocess import minmax_scaling
from train import train

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
config = {
    "learning_rate": 1e-4,
    "epochs": 500,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "window_size": 24,
    "output_size": 1,
    "dropout": 0.2,
    "patience": 10,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]

# LSTM

In [5]:
regions = ["Andong", "Seoul", "Jeonju", "Daegu"]

pred_list, loss_list = [], []

for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustDataset(df,
                            window_size=config["window_size"],
                            prediction_length=config["output_size"],
                            time_window=1)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  model = FinedustLSTM(input_size=len(dataset.feature_columns),
                              hidden_size=config['hidden_size'],
                              num_layers=config['num_layers'],
                              output_size=config['output_size'],
                              dropout_prob=config['dropout']).to(device)

  total_preds, losses = train(model,
                              train_dataset,
                              val_dataset,
                              dataset.feature_columns,
                              region, "LSTM", config,
                              device)
  pred_y = np.concatenate(total_preds, axis=0)
  val_y = np.concatenate([y for x, y in val_dataset])

  pred_list.append((pred_y, val_y))
  loss_list.append(losses)

/home/sangwon/nlplab_leaderboard2/Junho/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
/home/sangwon/anaconda3/envs/student/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 1/500 [00:02<18:14,  2.19s/it]

Epoch 1/500, Train Loss: 22.2224, Val Loss: 22.4068


  0%|          | 2/500 [00:04<17:07,  2.06s/it]

Epoch 2/500, Train Loss: 20.2971, Val Loss: 20.4735


  1%|          | 3/500 [00:06<16:45,  2.02s/it]

...

Epoch 145/500, Train Loss: 9.9040, Val Loss: 10.8489


 29%|██▉       | 146/500 [04:51<11:41,  1.98s/it]

Epoch 146/500, Train Loss: 10.0318, Val Loss: 11.2833


 29%|██▉       | 146/500 [04:53<11:50,  2.01s/it]

Epoch 147/500, Train Loss: 9.9619, Val Loss: 10.8485


* 전주와 광주에서 underfitting 발생

* hidden_size => 256

In [35]:
config = {
    "learning_rate": 1e-4,
    "epochs": 200,
    "batch_size": 32,
    "num_layers": 2,
    "hidden_size": 128,
    "window_size": 24,
    "output_size": 1,
    "dropout": 0.1,
    "patience": 10,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
region = "Gwangju"

if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
    
df = prepare_data(region.lower(), columns_to_remove)
df = simple_interpolate(df, method="linear")
df, scaler = minmax_scaling(df)

dataset = FinedustDataset(df,
                          window_size=config["window_size"],
                          prediction_length=config["output_size"],
                          time_window=1)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

model = FinedustLSTM(input_size=len(dataset.feature_columns),
                            hidden_size=config['hidden_size'],
                            num_layers=config['num_layers'],
                            output_size=config['output_size'],
                            dropout_prob=config['dropout']).to(device)

total_preds, losses = train(model,
                            train_dataset,
                            val_dataset,
                            dataset.feature_columns,
                            region, "LSTM", config,
                            device)
pred_y = np.concatenate(total_preds, axis=0)
val_y = np.concatenate([y for x, y in val_dataset])

/home/sangwon/nlplab_leaderboard2/Junho/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')
  0%|          | 1/200 [00:02<06:40,  2.01s/it]

Epoch 1/200, Train Loss: 30.3775, Val Loss: 24.3033


  1%|          | 2/200 [00:04<06:37,  2.01s/it]

Epoch 2/200, Train Loss: 29.4037, Val Loss: 24.2723


  2%|▏         | 3/200 [00:06<06:35,  2.01s/it]

...

Epoch 75/200, Train Loss: 23.4287, Val Loss: 24.2997


 38%|███▊      | 76/200 [02:34<04:12,  2.03s/it]

Epoch 76/200, Train Loss: 25.2223, Val Loss: 19.6870


 38%|███▊      | 76/200 [02:36<04:14,  2.06s/it]

Epoch 77/200, Train Loss: 23.3147, Val Loss: 19.3380


In [ ]:
region = "Gwangju"

if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
    
df = prepare_data(region.lower(), columns_to_remove)
df = simple_interpolate(df, method="linear")
df, scaler = minmax_scaling(df)

dataset = FinedustDataset(df,
                          window_size=config["window_size"],
                          prediction_length=config["output_size"],
                          time_window=1)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

model = FinedustLSTM(input_size=len(dataset.feature_columns),
                            hidden_size=config['hidden_size'],
                            num_layers=config['num_layers'],
                            output_size=config['output_size'],
                            dropout_prob=config['dropout']).to(device)

total_preds, losses = train(model,
                            train_dataset,
                            val_dataset,
                            dataset.feature_columns,
                            region, "LSTM", config,
                            device)
pred_y = np.concatenate(total_preds, axis=0)
val_y = np.concatenate([y for x, y in val_dataset])

In [36]:
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

def compute_permutation_importance(model, val_loader, feature_columns, device, loss_fn=nn.L1Loss()):
    model.eval()

    # Compute baseline performance
    baseline_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            preds = model(x)
            loss = loss_fn(preds, y)
            baseline_loss += loss.item() * x.size(0)
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y.cpu().numpy())

    baseline_loss /= len(val_loader.dataset)
    print(f"Baseline MSE Loss: {baseline_loss:.4f}")

    # Initialize importance dictionary
    feature_importances = {}

    # Convert validation data to a single tensor for manipulation
    all_x = []
    all_y = []
    for x, y in val_loader:
        all_x.append(x)
        all_y.append(y)
    all_x = torch.cat(all_x, dim=0).to(device)
    all_y = torch.cat(all_y, dim=0).to(device)

    for i, feature in enumerate(feature_columns):
        # Shuffle the i-th feature
        x_permuted = deepcopy(all_x)
        # Shuffle along the batch dimension
        perm = torch.randperm(x_permuted.size(0))
        x_permuted[:, :, i] = x_permuted[perm, :, i]

        # Compute loss with permuted feature
        preds = model(x_permuted)
        loss = loss_fn(preds, all_y).item()

        # Importance is the increase in loss
        importance = loss - baseline_loss
        feature_importances[feature] = importance
        print(f"Feature: {feature}, Permutation Importance: {importance:.4f}")

    return feature_importances

## Extract important features

In [38]:
regions

['Andong', 'Seoul', 'Jeonju', 'Daegu']

In [ ]:
output_dir = f"importance/LSTM"
os.makedirs(output_dir, exist_ok=True)

for region in regions:
  if region == "Jeonju":
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "IX", "PS", "지점", "위도", "경도"]
  else:
    columns_to_remove = ["CA_TOT", "CA_MID", "STN", "IR", "PA", "PS", "지점", "위도", "경도"]
  df = prepare_data(region.lower(), columns_to_remove)
  df = simple_interpolate(df, method="linear")
  df, scaler = minmax_scaling(df)

  dataset = FinedustDataset(df,
                            window_size=config["window_size"],
                            prediction_length=config["output_size"],
                            time_window=1)
  train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

  if region == "Jeonju":
    config["hidden_size"] = 256
  else:
    config["hidden_size"] = 128
  model = FinedustLSTM(input_size=len(dataset.feature_columns),
                              hidden_size=config['hidden_size'],
                              num_layers=config['num_layers'],
                              output_size=config['output_size'],
                              dropout_prob=config['dropout']).to(device)
  save_path = f"models/LSTM/{region}.pth"
  model.load_state_dict(torch.load(save_path))
  model.to(device)

  feature_importances = compute_permutation_importance(model, val_loader, dataset.feature_columns, device)
  output_path = f"{output_dir}/{region}.txt"

  with open(output_path, "w") as f:
    f.write("Feature Importances:\n")
    for feature, importance in sorted(feature_importances.items(), key=lambda x: x[1], reverse=True):
        f.write(f"{feature}: {importance:.4f}\n")
  print(f"Feature importances saved to {output_path}")

/home/sangwon/nlplab_leaderboard2/Junho/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')


Baseline MSE Loss: 6.4699
Feature: WD, Permutation Importance: 3.6131
Feature: WS, Permutation Importance: 3.5572
Feature: TA, Permutation Importance: 4.3083
Feature: TD, Permutation Importance: 1.9318
Feature: HM, Permutation Importance: 7.6094
Feature: PV, Permutation Importance: 4.6644
Feature: VS, Permutation Importance: 12.2706
Feature: TS, Permutation Importance: 3.0322
Feature importances saved to importance/LSTM/Andong.txt


/home/sangwon/nlplab_leaderboard2/Junho/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')


Baseline MSE Loss: 13.5851
Feature: WD, Permutation Importance: 6.2550
Feature: WS, Permutation Importance: 3.6059
Feature: TA, Permutation Importance: 1.1206
Feature: TD, Permutation Importance: 1.1049
Feature: HM, Permutation Importance: 5.1800
Feature: PV, Permutation Importance: 5.0815
Feature: VS, Permutation Importance: 10.2841
Feature: TS, Permutation Importance: 1.3686
Feature: TE_005, Permutation Importance: 0.7640
Feature: TE_01, Permutation Importance: 4.8736
Feature: TE_02, Permutation Importance: 5.2185
Feature: TE_03, Permutation Importance: 1.7073
Feature importances saved to importance/LSTM/Seoul.txt


/home/sangwon/nlplab_leaderboard2/Junho/JH/LSTM/interpolate.py:41: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data = data.interpolate(method='linear')


RuntimeError: Error(s) in loading state_dict for FinedustLSTM:
	size mismatch for lstm.lstm.weight_ih_l0: copying a param with shape torch.Size([1024, 8]) from checkpoint, the shape in current model is torch.Size([512, 8]).
	size mismatch for lstm.lstm.weight_hh_l0: copying a param with shape torch.Size([1024, 256]) from checkpoint, the shape in current model is torch.Size([512, 128]).
	size mismatch for lstm.lstm.bias_ih_l0: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for lstm.lstm.bias_hh_l0: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for lstm.lstm.weight_ih_l1: copying a param with shape torch.Size([1024, 256]) from checkpoint, the shape in current model is torch.Size([512, 128]).
	size mismatch for lstm.lstm.weight_hh_l1: copying a param with shape torch.Size([1024, 256]) from checkpoint, the shape in current model is

# Transformer

In [ ]:
import torch
from transformers import TimeSeriesTransformerForPrediction

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
plt.plot(val_y[:1000], label="val_y", color="blue")
plt.plot(pred_y[:1000], label="pred_y", color="orange")
plt.title(f"MLP (loss={np.min(losses[:, 1]): .2f})")
plt.legend()
plt.show()